In [ ]:
# tmdb爬蟲清整過資料 拉出 有日期 和 有片名 和 發行國家美國 和 有imdb_id 沒預算或沒票房 
# 留下['imdb_id','release_date','title_reg_new','budget','revenue','country_is_USA_new']直欄
# 新增year和checkid 直欄
#df = pd.read_json('movieresult.json')

cond1 = (df['release_date'] !='')&(df['title_reg_new'] != '')&(df['country_is_USA_new'] == True)&(df['imdb_id'] != '')&((df['budget'] == '')|(df['revenue'] == ''))
df_new = df[cond1].copy()

#df_new.to_json('ng_homework.json')

dfsort = df_new[['imdb_id','release_date','title_reg_new','budget','revenue','country_is_USA_new']].copy()
dfsort['year'] = dfsort['release_date'].str[:4]
dfsort['title_reg_new'] = dfsort['title_reg_new'].str.lower() 
dfsort['year'] = dfsort['year'].astype('int')
dfsort['checkid'] = np.arange(dfsort.shape[0])

# imdb資料清整 有關錢的直欄去除 $ 和 , 轉成int格式
# 發行日期Unknow的丟掉
# 新增checkid 直欄

df_number = pd.read_json('the_numbers.json')
df_number['ProductionBudget'] = df_number['ProductionBudget'].str.replace('$','').str.replace(',','')
df_number['DomesticGross'] = df_number['DomesticGross'].str.replace('$','').str.replace(',','')
df_number['WorldwideGross'] = df_number['WorldwideGross'].str.replace('$','').str.replace(',','')
df_number['year'] = df_number['ReleaseDate'].str[-4:]
df_number['Movie'] = df_number['Movie'].str.replace('[^a-zA-Z0-9 ]',' ',regex = True).str.replace(' +',' ',regex = True).str.extract('([ \dA-Za-z]*)')
df_number = df_number[df_number['ReleaseDate'] != 'Unknown']
df_number['Movie'] = df_number['Movie'].str.lower()
df_number['year'] = df_number['year'].astype('int')
df_number['checkid'] = np.arange(df_number.shape[0])

# 篩出兩張表有電影名稱相符的資料

dfsort2 = dfsort[dfsort['title_reg_new'].isin(df_number['Movie'])]
dfnumber2 = df_number[df_number['Movie'].isin(dfsort['title_reg_new'])]

# 篩出電影名稱相同 發行年分差距 +/- 1年的電影資料

dfsort2_listindex = []
dfnumber2_listindex = []
for checkid in dfsort2['checkid'] :
    moviename = dfsort2.loc[dfsort2['checkid'] == checkid,'title_reg_new'].values[0]
    targetyear= dfsort2.loc[dfsort2['checkid'] == checkid,'year'].values[0]
    judgeyear = dfnumber2.loc[dfnumber2['Movie'] == moviename,'year'].values[0]
    if (judgeyear -1 <= targetyear) &  (targetyear <= judgeyear + 1) :
        dfsort2_correctid = checkid
        dfnumber2_correctid = dfnumber2.loc[dfnumber2['Movie'] == moviename,'checkid'].values[0]
        # 去重 重複id 跳過
        if dfnumber2_correctid in dfnumber2_listindex:
            continue
        dfsort2_listindex.append(checkid)
        dfnumber2_listindex.append(dfnumber2_correctid)

# 依上述checkid 拿到兩張表格 資料匹配的表格        
dfsort3 = dfsort2.loc[dfsort2['checkid'].isin(dfsort2_listindex)].copy()
dfnumber3 = dfnumber2.loc[dfnumber2['checkid'].isin(dfnumber2_listindex)].copy()

# join key = Movie

dfsum = pd.merge(dfsort3, dfnumber3,how ='inner',left_on='title_reg_new',right_on='Movie').copy()

# 票房和預算有'' 轉成0 , 值欄型態改為int

dfsum.loc[dfsum['budget'] == '' , 'budget'] = 0
dfsum.loc[dfsum['revenue'] == '' , 'revenue'] = 0
dfsum.loc[dfsum['ProductionBudget'] == '' , 'ProductionBudget'] = 0
dfsum.loc[dfsum['DomesticGross'] == '' , 'DomesticGross'] = 0
dfsum['budget'] = dfsum['budget'].astype('int')
dfsum['revenue'] = dfsum['revenue'].astype('int')
dfsum['ProductionBudget'] = dfsum['ProductionBudget'].astype('int')
dfsum['DomesticGross'] = dfsum['DomesticGross'].astype('int')

# 補值budget 和 revenue

dfsum.loc[dfsum['budget'] == 0 , 'budget'] = dfsum.loc[dfsum['budget'] == 0,'ProductionBudget']
dfsum.loc[dfsum['revenue'] == 0 , 'revenue'] = dfsum.loc[dfsum['revenue'] == 0,'DomesticGross']

# 補值完預算和票房不為0的資料存成json檔

dfsum.loc[(dfsum['budget'] != 0) & (dfsum['revenue'] != 0),['imdb_id','release_date','title_reg_new','budget','revenue']].to_json('check2movie.json')